Импортирую кучу библиотек

In [39]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
import matplotlib.pyplot as plt

Загружаю датасет

In [41]:
df = pd.read_csv('bbc-news-data.csv', sep =('\t'))
df.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


Создаю функцию для обработки данных.
Привожу данные к нижнему регистру, удаляю знаки препинания и символы, разбиваю на слова, удалю стоп-слова, извлекаю корни слов, объединяю слова в один текст.

In [44]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
ps = nltk.PorterStemmer()

def preprocess(content):
    content = content.lower()
    content = re.sub('[^a-zA-Z]', ' ', content)
    words = content.split()
    words = [word for word in words if word not in stop_words]
    words = [ps.stem(word) for word in words]
    content = ' '.join(words)
    return content

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Предварительно обрабатываю текст

In [45]:
df["content"] = df['content'].apply(preprocess)

Разделяю датасет на тестовый и тренировчный

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df["content"], df["category"], test_size=0.2, random_state=42)

Создаю модель

In [48]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB())
])

Определяю гиперпараметры и их значения

In [49]:
parameters = {
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
    'clf__alpha': [0.1, 0.5, 1],
}

Определяю гиперпараметр для построения кривой валидации

In [50]:
parameter = 'tfidf__ngram_range'

Создаю GridSearchCV

In [51]:
search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)

Обучаю модель GridSearchCV

In [52]:
search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.1, 0.5, 1],
                         'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             verbose=1)

Вывожу самые лучшие гиперпараметры

In [53]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.977):
{'clf__alpha': 0.1, 'tfidf__ngram_range': (1, 2)}
